In [1]:
import altair as alt
import pandas as pd

In [2]:
def calc_total_bp(read_length=100, reads=1000000, paired=True):
    """
    Calculates total basepairs sequenced

    Parameters
    ----------
    read_length : int
        Length of sequenced reads.
    reads : int
        Total number of sequenced reads.
    paired : boolean
        Is the sequencing paired-end?
    
    Returns
    -------
    total_bp : int
        Total basepairs sequenced.
    """
    total_bp = read_length * reads
    if paired:
        total_bp = total_bp * 2
    return total_bp


def create_plotdata(min_reads=10, max_reads=100, step_reads=10, read_length=100, paired=True, 
                    min_genome_size=1, max_genome_size=5, step_genome_size=1,
                    min_percent=1, max_percent=100, step_percent=1):
    """
    Create data for plotting.

    This creates a long-format dataframe with the columns '%', 'reads' (in
    million), 'genome_size' (in Mbp) and 'coverage'. The coverage for each row
    shows the estimated coverage of a genome of size 'genome_size', present at
    '%' in the sample given 'reads' million reads sequenced.

    Parameters
    ----------
    min_reads : int
        Minimum number of reads (in millions) to include in dataframe.
    max_reads : int
        Maximum number of reads (in millions) to include in dataframe.
    step_reads : int
        Increment of reads (in millions) to use to construct the range.
    read_length : int
        Length of reads (in bp).
    paired : boolean
        Whether paired-end sequencing is performed.
    min_genome_size : int
        Minimum genome size (in Mbp) to include in dataframe.
    max_genome_size : int
        Maximum genome size (in Mbp) to include in dataframe.
    step_genome_size : int
        Increment of genome size (in Mbp).
    min_percent : int
        Minimum percent to include in dataframe.
    max_percent : int
        Maximum percent to include in dataframe.
    step_percent : int
        Increment of percent.
    
    Returns
    -------
    pandas.DataFrame
        A dataframe in long-format.
    """
    # Create the ranges
    reads_range = range(min_reads, max_reads+step_reads, step_reads)
    genome_size_range = range(min_genome_size, max_genome_size+step_genome_size, step_genome_size)
    percent_range = range(min_percent, max_percent+step_percent, step_percent)
    # Initialize results as dictionary
    res = {"%": [], "reads": [], "genome_size": [], "coverage": []}
    # Loop over reads in reads range
    for reads in reads_range:
        # Scale by 1 million (for actual calculations)
        _reads = reads*1000000
        # Calculate total basepairs
        total_bp = calc_total_bp(read_length=read_length, reads=_reads, paired=paired)
        # Loop over genome sizes
        for genome_size in genome_size_range:
            # Scale by 1 million (for actual calculations)
            _genome_size = genome_size*1000000
            # Loop over percent in percent range
            for percent in percent_range:
                # Scale to fraction (for actual calculations)
                _percent = percent / 100
                # Calculate coverage by dividing total basepairs with size of genome and scaling by percent
                covX = (total_bp / _genome_size)*(_percent)
                # Add results to dictionary
                res["%"].append(percent)
                res["reads"].append(f"{reads}M")
                res["genome_size"].append(f"{genome_size}Mbp")
                res["coverage"].append(covX)
    return pd.DataFrame(res)


def make_plot(plotdata):
    """
    Creates an altair plot.

    Parameters
    ----------
    plotdata : pandas.DataFrame
        The long format dataframe used for plotting
    
    Returns
    -------
    altair chart
        The altair chart
    """
    p = alt.Chart(plotdata).mark_point(strokeWidth=.1, size=30).encode(
        x=alt.X("%", title="Species relative abundance (%)"), 
        y=alt.Y("coverage", title="Genome coverage").scale(type="log"),
        fill=alt.Fill("reads", sort=plotdata.reads.unique()),
        shape="genome_size",
        tooltip=["%", "reads", "genome_size", "coverage"]
    )
    return p

In [3]:
# Create plotdata using default values
plotdata = create_plotdata()

# Plot altair chart
make_plot(plotdata)

alt.Chart(...)

In [4]:
# Same plot but with radio buttons for genome size for increased visibility
plotdata = create_plotdata()

genome_size_options = list(plotdata.genome_size.unique())
genome_size_labels = [option + ' ' for option in genome_size_options]

# Create radio button bindings
genome_size_radio_buttons = alt.binding_radio(
    # Add the empty selection which shows all when clicked
    options=genome_size_options + [None],
    labels=genome_size_labels + ['All'],
    name='Genome size: '
)

# Create a selection filter
genome_size_selection = alt.selection_point(
    fields=['genome_size'],
    value="1Mbp",
    bind=genome_size_radio_buttons,
)

# Plot again and add radio buttons
make_plot(plotdata).add_params(
    genome_size_selection
).transform_filter(
    genome_size_selection
)

alt.Chart(...)